In [ ]:
import os
import pandas as pd
import pandas_ta as ta


In [ ]:
MINIMUM_TIME = 930
MAXIMUM_TIME = 1430

MINIMUM_DATE_DATAFRAME = 20200401
MINIMUM_DATE_TRADE = 20200501

CURRENT_TARGET = 0.6
CURRENT_STOP = 0.2

In [ ]:
csv_diario = pd.read_csv(os.path.join('..','00.data','input','WDO$Daily.csv'))
csv_5Min = pd.read_csv(os.path.join('..','00.data','input','WDO$M5.csv'))

csv_diario['Date'] = csv_diario.apply(lambda x: int(x['datetime'].replace('.','')), axis=1)
csv_5Min['Date'] = csv_5Min.apply(lambda x: int(x['datetime'].split(' ')[0].replace('.','')), axis=1)
csv_5Min['Time'] = csv_5Min.apply(lambda x: int(x['datetime'].split(' ')[1].replace(':','')), axis=1)

csv_diario = csv_diario[csv_diario['Date'] >= MINIMUM_DATE_DATAFRAME]
csv_5Min = csv_5Min[csv_5Min['Date'] >= MINIMUM_DATE_DATAFRAME]

csv_5Min['ema9'] = csv_5Min.ta.ema(length=9)
csv_5Min['ema21'] = csv_5Min.ta.ema(length=21)

#csv_5Min.dtypes
#csv_5Min.head()

In [ ]:
data_daily = csv_diario.to_dict('records')
data_5min = csv_5Min.to_dict('records')

ohlc_daily = {}

for current_date in data_daily:
    ohlc_daily[current_date['Date']] = current_date

print(data_5min[:15])

In [ ]:
generated_bars = []

def is_short(bar, target, stop):
    correct_target = bar['x5_close'] - ((target))
    correct_stop = bar['x5_close'] + ((stop))
    return (min(bar['y0_low'], bar['y1_low'], bar['y2_low'], bar['y3_low'], bar['y4_low'], bar['y5_low']) < correct_target) and (max(bar['y0_high'], bar['y1_high'], bar['y2_high'], bar['y3_high'], bar['y4_high'], bar['y5_high']) < correct_stop)
    
def is_long(bar, target, stop):
    correct_target = bar['x5_close'] + ((target))
    correct_stop = bar['x5_close'] -  ((stop))
    return (max(bar['y0_high'], bar['y1_high'], bar['y2_high'], bar['y3_high'], bar['y4_high'], bar['y5_high']) > correct_target) and (min(bar['y0_low'], bar['y1_low'], bar['y2_low'], bar['y3_low'], bar['y4_low'], bar['y5_low']) < correct_stop)


for index, _ in enumerate(data_5min):
    if data_5min[index]['Time'] < MINIMUM_TIME or \
        data_5min[index]['Time'] > MAXIMUM_TIME or \
        data_5min[index]['Date'] < MINIMUM_DATE:
        continue
        
    current_open = data_5min[index - 5]['open']
    current_volume = data_5min[index - 5]['volume']
    
    current_bar = {}
    
    current_bar['current_open'] = current_open
    
    
    current_bar['x0_ema9'] = ((data_5min[index-5]['ema9'] / current_open) - 1) * 100
    current_bar['x0_ema21'] = ((data_5min[index-5]['ema21'] / current_open) - 1) * 100
    current_bar['x0_open'] = ((data_5min[index-5]['open'] / current_open) - 1) * 100
    current_bar['x0_high'] = ((data_5min[index-5]['high'] / current_open) - 1) * 100
    current_bar['x0_low'] = ((data_5min[index-5]['low'] / current_open) - 1) * 100
    current_bar['x0_close'] = ((data_5min[index-5]['close'] / current_open) - 1) * 100
    current_bar['x0_volume'] = ((data_5min[index-5]['volume'] / current_open) - 1) * 100
    current_bar['x0_open_real'] = data_5min[index-5]['open'] 
    current_bar['x0_high_real'] = data_5min[index-5]['high'] 
    current_bar['x0_low_real'] = data_5min[index-5]['low'] 
    current_bar['x0_close_real'] = data_5min[index-5]['close'] 

    
    current_bar['x1_ema9'] = ((data_5min[index-4]['ema9'] / current_open) - 1) * 100
    current_bar['x1_ema21'] = ((data_5min[index-4]['ema21'] / current_open) - 1) * 100
    current_bar['x1_open'] = ((data_5min[index-4]['open'] / current_open) - 1) * 100
    current_bar['x1_high'] = ((data_5min[index-4]['high'] / current_open) - 1) * 100
    current_bar['x1_low'] = ((data_5min[index-4]['low'] / current_open) - 1) * 100
    current_bar['x1_close'] = ((data_5min[index-4]['close'] / current_open) - 1) * 100
    current_bar['x1_volume'] = ((data_5min[index-4]['volume'] / current_open) - 1) * 100
    current_bar['x1_open_real'] = data_5min[index-4]['open'] 
    current_bar['x1_high_real'] = data_5min[index-4]['high'] 
    current_bar['x1_low_real'] = data_5min[index-4]['low'] 
    current_bar['x1_close_real'] = data_5min[index-4]['close'] 

    
    current_bar['x2_ema9'] = ((data_5min[index-3]['ema9'] / current_open) - 1) * 100
    current_bar['x2_ema21'] = ((data_5min[index-3]['ema21'] / current_open) - 1) * 100
    current_bar['x2_open'] = ((data_5min[index-3]['open'] / current_open) - 1) * 100
    current_bar['x2_high'] = ((data_5min[index-3]['high'] / current_open) - 1) * 100
    current_bar['x2_low'] = ((data_5min[index-3]['low'] / current_open) - 1) * 100
    current_bar['x2_close'] = ((data_5min[index-3]['close'] / current_open) - 1) * 100
    current_bar['x2_volume'] = ((data_5min[index-3]['volume'] / current_open) - 1) * 100    
    current_bar['x2_open_real'] = data_5min[index-3]['open'] 
    current_bar['x2_high_real'] = data_5min[index-3]['high'] 
    current_bar['x2_low_real'] = data_5min[index-3]['low'] 
    current_bar['x2_close_real'] = data_5min[index-3]['close']

    
    current_bar['x3_ema9'] = ((data_5min[index-2]['ema9'] / current_open) - 1) * 100
    current_bar['x3_ema21'] = ((data_5min[index-2]['ema21'] / current_open) - 1) * 100
    current_bar['x3_open'] = ((data_5min[index-2]['open'] / current_open) - 1) * 100
    current_bar['x3_high'] = ((data_5min[index-2]['high'] / current_open) - 1) * 100
    current_bar['x3_low'] = ((data_5min[index-2]['low'] / current_open) - 1) * 100
    current_bar['x3_close'] = ((data_5min[index-2]['close'] / current_open) - 1) * 100
    current_bar['x3_volume'] = ((data_5min[index-2]['volume'] / current_open) - 1) * 100        
    current_bar['x3_open_real'] = data_5min[index-2]['open'] 
    current_bar['x3_high_real'] = data_5min[index-2]['high'] 
    current_bar['x3_low_real'] = data_5min[index-2]['low'] 
    current_bar['x3_close_real'] = data_5min[index-2]['close']

    
    current_bar['x4_ema9'] = ((data_5min[index-1]['ema9'] / current_open) - 1) * 100
    current_bar['x4_ema21'] = ((data_5min[index-1]['ema21'] / current_open) - 1) * 100
    current_bar['x4_open'] = ((data_5min[index-1]['open'] / current_open) - 1) * 100
    current_bar['x4_high'] = ((data_5min[index-1]['high'] / current_open) - 1) * 100
    current_bar['x4_low'] = ((data_5min[index-1]['low'] / current_open) - 1) * 100
    current_bar['x4_close'] = ((data_5min[index-1]['close'] / current_open) - 1) * 100
    current_bar['x4_volume'] = ((data_5min[index-1]['volume'] / current_open) - 1) * 100            
    current_bar['x4_open_real'] = data_5min[index-1]['open'] 
    current_bar['x4_high_real'] = data_5min[index-1]['high'] 
    current_bar['x4_low_real'] = data_5min[index-1]['low'] 
    current_bar['x4_close_real'] = data_5min[index-1]['close'] 

    
    current_bar['x5_ema9'] = ((data_5min[index]['ema9'] / current_open) - 1) * 100
    current_bar['x5_ema21'] = ((data_5min[index]['ema21'] / current_open) - 1) * 100
    current_bar['x5_open'] = ((data_5min[index]['open'] / current_open) - 1) * 100
    current_bar['x5_high'] = ((data_5min[index]['high'] / current_open) - 1) * 100
    current_bar['x5_low'] = ((data_5min[index]['low'] / current_open) - 1) * 100
    current_bar['x5_close'] = ((data_5min[index]['close'] / current_open) - 1) * 100
    current_bar['x5_volume'] = ((data_5min[index]['volume'] / current_open) - 1) * 100            
    current_bar['x5_open_real'] = data_5min[index]['open'] 
    current_bar['x5_high_real'] = data_5min[index]['high'] 
    current_bar['x5_low_real'] = data_5min[index]['low'] 
    current_bar['x5_close_real'] = data_5min[index]['close'] 

    
    current_bar['y0_ema9'] = ((data_5min[index+1]['ema9'] / current_open) - 1) * 100
    current_bar['y0_ema21'] = ((data_5min[index+1]['ema21'] / current_open) - 1) * 100
    current_bar['y0_open'] = ((data_5min[index+1]['open'] / current_open) - 1) * 100
    current_bar['y0_high'] = ((data_5min[index+1]['high'] / current_open) - 1) * 100
    current_bar['y0_low'] = ((data_5min[index+1]['low'] / current_open) - 1) * 100
    current_bar['y0_close'] = ((data_5min[index+1]['close'] / current_open) - 1) * 100
    current_bar['y0_volume'] = ((data_5min[index+1]['volume'] / current_open) - 1) * 100
    current_bar['y0_open_real'] = data_5min[index+1]['open'] 
    current_bar['y0_high_real'] = data_5min[index+1]['high'] 
    current_bar['y0_low_real'] = data_5min[index+1]['low'] 
    current_bar['y0_close_real'] = data_5min[index+1]['close'] 

    
    current_bar['y1_ema9'] = ((data_5min[index+2]['ema9'] / current_open) - 1) * 100
    current_bar['y1_ema21'] = ((data_5min[index+2]['ema21'] / current_open) - 1) * 100
    current_bar['y1_open'] = ((data_5min[index+2]['open'] / current_open) - 1) * 100
    current_bar['y1_high'] = ((data_5min[index+2]['high'] / current_open) - 1) * 100
    current_bar['y1_low'] = ((data_5min[index+2]['low'] / current_open) - 1) * 100
    current_bar['y1_close'] = ((data_5min[index+2]['close'] / current_open) - 1) * 100
    current_bar['y1_volume'] = ((data_5min[index+2]['volume'] / current_open) - 1) * 100
    current_bar['y1_open_real'] = data_5min[index+2]['open'] 
    current_bar['y1_high_real'] = data_5min[index+2]['high'] 
    current_bar['y1_low_real'] = data_5min[index+2]['low'] 
    current_bar['y1_close_real'] = data_5min[index+2]['close'] 
    
    current_bar['y2_ema9'] = ((data_5min[index+3]['ema9'] / current_open) - 1) * 100
    current_bar['y2_ema21'] = ((data_5min[index+3]['ema21'] / current_open) - 1) * 100
    current_bar['y2_open'] = ((data_5min[index+3]['open'] / current_open) - 1) * 100
    current_bar['y2_high'] = ((data_5min[index+3]['high'] / current_open) - 1) * 100
    current_bar['y2_low'] = ((data_5min[index+3]['low'] / current_open) - 1) * 100
    current_bar['y2_close'] = ((data_5min[index+3]['close'] / current_open) - 1) * 100
    current_bar['y2_volume'] = ((data_5min[index+3]['volume'] / current_open) - 1) * 100    
    current_bar['y2_open_real'] = data_5min[index+3]['open'] 
    current_bar['y2_high_real'] = data_5min[index+3]['high'] 
    current_bar['y2_low_real'] = data_5min[index+3]['low'] 
    current_bar['y2_close_real'] = data_5min[index+3]['close'] 

    
    current_bar['y3_ema9'] = ((data_5min[index+4]['ema9'] / current_open) - 1) * 100
    current_bar['y3_ema21'] = ((data_5min[index+4]['ema21'] / current_open) - 1) * 100
    current_bar['y3_open'] = ((data_5min[index+4]['open'] / current_open) - 1) * 100
    current_bar['y3_high'] = ((data_5min[index+4]['high'] / current_open) - 1) * 100
    current_bar['y3_low'] = ((data_5min[index+4]['low'] / current_open) - 1) * 100
    current_bar['y3_close'] = ((data_5min[index+4]['close'] / current_open) - 1) * 100
    current_bar['y3_volume'] = ((data_5min[index+4]['volume'] / current_open) - 1) * 100        
    current_bar['y3_open_real'] = data_5min[index+4]['open'] 
    current_bar['y3_high_real'] = data_5min[index+4]['high'] 
    current_bar['y3_low_real'] = data_5min[index+4]['low'] 
    current_bar['y3_close_real'] = data_5min[index+4]['close']

    
    current_bar['y4_ema9'] = ((data_5min[index+5]['ema9'] / current_open) - 1) * 100
    current_bar['y4_ema21'] = ((data_5min[index+5]['ema21'] / current_open) - 1) * 100
    current_bar['y4_open'] = ((data_5min[index+5]['open'] / current_open) - 1) * 100
    current_bar['y4_high'] = ((data_5min[index+5]['high'] / current_open) - 1) * 100
    current_bar['y4_low'] = ((data_5min[index+5]['low'] / current_open) - 1) * 100
    current_bar['y4_close'] = ((data_5min[index+5]['close'] / current_open) - 1) * 100
    current_bar['y4_volume'] = ((data_5min[index+5]['volume'] / current_open) - 1) * 100            
    current_bar['y4_open_real'] = data_5min[index+5]['open'] 
    current_bar['y4_high_real'] = data_5min[index+5]['high'] 
    current_bar['y4_low_real'] = data_5min[index+5]['low'] 
    current_bar['y4_close_real'] = data_5min[index+5]['close'] 

    
    current_bar['y5_ema9'] = ((data_5min[index+6]['ema9'] / current_open) - 1) * 100
    current_bar['y5_ema21'] = ((data_5min[index+6]['ema21'] / current_open) - 1) * 100
    current_bar['y5_open'] = ((data_5min[index+6]['open'] / current_open) - 1) * 100
    current_bar['y5_high'] = ((data_5min[index+6]['high'] / current_open) - 1) * 100
    current_bar['y5_low'] = ((data_5min[index+6]['low'] / current_open) - 1) * 100
    current_bar['y5_close'] = ((data_5min[index+6]['close'] / current_open) - 1) * 100
    current_bar['y5_volume'] = ((data_5min[index+6]['volume'] / current_open) - 1) * 100                
    current_bar['y5_open_real'] = data_5min[index+6]['open'] 
    current_bar['y5_high_real'] = data_5min[index+6]['high']
    current_bar['y5_low_real'] = data_5min[index+6]['low']
    current_bar['y5_close_real'] = data_5min[index+6]['close']

    
    

    current_bar['chart_data'] = [
        {
        'date': 0,
        'open': current_bar['x0_open'],
        'high': current_bar['x0_high'],
        'low': current_bar['x0_low'],
        'close': current_bar['x0_close'],
        'ema9': current_bar['x0_ema9'],
        'ema21': current_bar['x0_ema21'],
        'open_real': current_bar['x0_open_real'],
        'high_real': current_bar['x0_high_real'],
        'low_real': current_bar['x0_low_real'],
        'close_real': current_bar['x0_close_real']
        },
        {
        'date': 1,
        'open': current_bar['x1_open'],
        'high': current_bar['x1_high'],
        'low': current_bar['x1_low'],
        'close': current_bar['x1_close'],
        'ema9': current_bar['x1_ema9'],
        'ema21': current_bar['x1_ema21'],
        'open_real': current_bar['x1_open_real'],
        'high_real': current_bar['x1_high_real'],
        'low_real': current_bar['x1_low_real'],
        'close_real': current_bar['x1_close_real']
        },
        {
        'date': 2,
        'open': current_bar['x2_open'],
        'high': current_bar['x2_high'],
        'low': current_bar['x2_low'],
        'close': current_bar['x2_close'],
        'ema9': current_bar['x2_ema9'],
        'ema21': current_bar['x2_ema21'],
        'open_real': current_bar['x2_open_real'],
        'high_real': current_bar['x2_high_real'],
        'low_real': current_bar['x2_low_real'],
        'close_real': current_bar['x2_close_real']
        },
        {
        'date': 3,
        'open': current_bar['x3_open'],
        'high': current_bar['x3_high'],
        'low': current_bar['x3_low'],
        'close': current_bar['x3_close'],
        'ema9': current_bar['x3_ema9'],
        'ema21': current_bar['x3_ema21'],
        'open_real': current_bar['x3_open_real'],
        'high_real': current_bar['x3_high_real'],
        'low_real': current_bar['x3_low_real'],
        'close_real': current_bar['x3_close_real']
        },     
        {
        'date': 4,
        'open': current_bar['x4_open'],
        'high': current_bar['x4_high'],
        'low': current_bar['x4_low'],
        'close': current_bar['x4_close'],
        'ema9': current_bar['x4_ema9'],
        'ema21': current_bar['x4_ema21'],
        'open_real': current_bar['x4_open_real'],
        'high_real': current_bar['x4_high_real'],
        'low_real': current_bar['x4_low_real'],
        'close_real': current_bar['x4_close_real']
        },             
        {
        'date': 5,
        'open': current_bar['x5_open'],
        'high': current_bar['x5_high'],
        'low': current_bar['x5_low'],
        'close': current_bar['x5_close'],
        'ema9': current_bar['x5_ema9'],
        'ema21': current_bar['x5_ema21'],
        'open_real': current_bar['x5_open_real'],
        'high_real': current_bar['x5_high_real'],
        'low_real': current_bar['x5_low_real'],
        'close_real': current_bar['x5_close_real']
        },                     
        {
        'date': 6,
        'open': current_bar['y0_open'],
        'high': current_bar['y0_high'],
        'low': current_bar['y0_low'],
        'close': current_bar['y0_close'],
        'ema9': current_bar['y0_ema9'],
        'ema21': current_bar['y0_ema21'],
        'open_real': current_bar['y0_open_real'],
        'high_real': current_bar['y0_high_real'],
        'low_real': current_bar['y0_low_real'],
        'close_real': current_bar['y0_close_real']
        },
        {
        'date': 7,
        'open': current_bar['y1_open'],
        'high': current_bar['y1_high'],
        'low': current_bar['y1_low'],
        'close': current_bar['y1_close'],
        'ema9': current_bar['y1_ema9'],
        'ema21': current_bar['y1_ema21'],           
        'open_real': current_bar['y1_open_real'],
        'high_real': current_bar['y1_high_real'],
        'low_real': current_bar['y1_low_real'],
        'close_real': current_bar['y1_close_real']
        },
        {
        'date': 8,
        'open': current_bar['y2_open'],
        'high': current_bar['y2_high'],
        'low': current_bar['y2_low'],
        'close': current_bar['y2_close'],
        'ema9': current_bar['y2_ema9'],
        'ema21': current_bar['y2_ema21'],
        'open_real': current_bar['y2_open_real'],
        'high_real': current_bar['y2_high_real'],
        'low_real': current_bar['y2_low_real'],
        'close_real': current_bar['y2_close_real']
        },
        {
        'date': 9,
        'open': current_bar['y3_open'],
        'high': current_bar['y3_high'],
        'low': current_bar['y3_low'],
        'close': current_bar['y3_close'],
        'ema9': current_bar['y3_ema9'],
        'ema21': current_bar['y3_ema21'],
        'open_real': current_bar['y3_open_real'],
        'high_real': current_bar['y3_high_real'],
        'low_real': current_bar['y3_low_real'],
        'close_real': current_bar['y3_close_real']
        },     
        {
        'date': 10,
        'open': current_bar['y4_open'],
        'high': current_bar['y4_high'],
        'low': current_bar['y4_low'],
        'close': current_bar['y4_close'],
        'ema9': current_bar['y4_ema9'],
        'ema21': current_bar['y4_ema21'],
        'open_real': current_bar['y4_open_real'],
        'high_real': current_bar['y4_high_real'],
        'low_real': current_bar['y4_low_real'],
        'close_real': current_bar['y4_close_real']
        },             
        {
        'date': 11,
        'open': current_bar['y5_open'],
        'high': current_bar['y5_high'],
        'low': current_bar['y5_low'],
        'close': current_bar['y5_close'],
        'ema9': current_bar['y5_ema9'],
        'ema21': current_bar['y5_ema21'], 
        'open_real': current_bar['y5_open_real'],
        'high_real': current_bar['y5_high_real'],
        'low_real': current_bar['y5_low_real'],
        'close_real': current_bar['y5_close_real']
        }]
    
    current_bar['current_pos'] = current_bar['x5_close']
    current_bar['is_short'] = is_short(current_bar, CURRENT_TARGET, CURRENT_STOP)
    current_bar['is_long'] = is_long(current_bar, CURRENT_TARGET, CURRENT_STOP)
    
    generated_bars.append(current_bar)
    
generated_bars = [x for x in generated_bars if x['is_short'] == True]

df = pd.DataFrame(generated_bars[0]['chart_data'])

print(generated_bars[0]['current_pos'])
print(df)
    
    
    

In [ ]:
from math import pi
from bokeh.plotting import figure, show

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

p = figure(x_axis_type="linear", tools=TOOLS, width=1000, title = "WDO Candlestick")
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha=0.3
w = 12*60*60*1000 # half day in ms

inc = df.close > df.open
dec = df.open > df.close

p.segment(df.date, df.high, df.date, df.low, color="black")
p.vbar(df.date[inc], 0.8, df.open[inc], df.close[inc], fill_color="#D5E1DD", line_color="black")
p.vbar(df.date[dec], 0.8, df.open[dec], df.close[dec], fill_color="#F2583E", line_color="black")

p.line(df.date, df.ema9, line_color="#00FF00", line_width = 5)
p.line(df.date, df.ema21, line_color="#FFFF00", line_width = 5)

show(p)